In [1]:
import requests,json,datetime
from urllib.parse import quote 
import pandas as pd
import numpy as np
import folium

In [2]:
with open('data/sub_arr_info_key.txt') as f:
    info_key = f.read()

In [10]:
df_st = pd.read_csv('data/St_addr_20221122.csv',encoding='euc-kr')
df_st = df_st[['역명','도로명주소']]

In [4]:
target = input('역명 입력')
target = target[:-1] if target[-1] == '역' else target

In [6]:
base_url = "http://swopenAPI.seoul.go.kr/api/subway/"
params1 = f"{info_key}/json/realtimeStationArrival/0/8/"
params2 = quote(target)
url = f"{base_url}{params1}{params2}"
response = requests.get(url)
res = json.loads(response.text)

In [7]:
df = pd.DataFrame(res['realtimeArrivalList'])
df = df[['updnLine','trainLineNm','statnNm','bstatnNm','arvlMsg2','arvlMsg3','barvlDt','arvlCd']]

In [8]:
df

,updnLine,trainLineNm,statnNm,bstatnNm,arvlMsg2,arvlMsg3,barvlDt,arvlCd
0,상행,광운대행 - 화서방면,수원,광운대,수원 진입,세류,0,0
1,상행,청량리행 - 화서방면 (급행),수원,청량리 (급행),[2]번째 전역 (병점),병점,0,99
2,상행,왕십리행 - 매교방면,수원,왕십리,[3]번째 전역 (어천),어천,0,99
3,상행,광운대행 - 화서방면,수원,광운대,[4]번째 전역 (오산대),오산대,0,99
4,상행,왕십리행 - 매교방면,수원,왕십리,[11]번째 전역 (신길온천),신길온천,0,99
5,상행,청량리행 - 화서방면 (급행),수원,청량리 (급행),[13]번째 전역 (두정),두정,0,99
6,하행,서동탄행 - 세류방면,수원,서동탄,전역 진입,화서,0,4
7,하행,고색행 - 고색방면,수원,고색,[2]번째 전역 (수원시청),수원시청,0,99


In [9]:
df1 = df[df.updnLine == '상행'].copy()
df2 = df[df.updnLine == '하행'].copy()

In [11]:
def kakao_location(place):
    with open('../04.지도시각화/data/kakaoapikey.txt') as f_:
        kakao_key = f_.read()
    base_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{base_url}?query={quote(place)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lat_ = float(result['documents'][0]['y'])
    lng_ = float(result['documents'][0]['x'])
    return lat_,lng_

In [12]:
def rtn_addr(target):
    str_addr = df_st[df_st.역명 == target].도로명주소.values[-1]
    return str_addr.strip()

In [13]:
temp1 =[]
for i in df1.index:
    bst = df1.arvlMsg3[i].strip()
    temp1.append(kakao_location(rtn_addr(bst)))

df_test = pd.DataFrame(temp1,columns=('lat','lng'))
df1 = pd.concat([df1, df_test], axis=1)
df1

,updnLine,trainLineNm,statnNm,bstatnNm,arvlMsg2,arvlMsg3,barvlDt,arvlCd,lat,lng
0,상행,광운대행 - 화서방면,수원,광운대,수원 진입,세류,0,0,37.244655,127.013978
1,상행,청량리행 - 화서방면 (급행),수원,청량리 (급행),[2]번째 전역 (병점),병점,0,99,37.206802,127.033243
2,상행,왕십리행 - 매교방면,수원,왕십리,[3]번째 전역 (어천),어천,0,99,37.250119,126.908786
3,상행,광운대행 - 화서방면,수원,광운대,[4]번째 전역 (오산대),오산대,0,99,37.169301,127.063065
4,상행,왕십리행 - 매교방면,수원,왕십리,[11]번째 전역 (신길온천),신길온천,0,99,37.337579,126.767149
5,상행,청량리행 - 화서방면 (급행),수원,청량리 (급행),[13]번째 전역 (두정),두정,0,99,36.832935,127.149021


In [15]:
temp1 =[]
for i in df2.index:
    try:
        bst = df2.arvlMsg3[i].strip()
        temp1.append(kakao_location(rtn_addr(bst)))
    except:
        print(df2.arvlMsg3[i].strip())

df_test = pd.DataFrame(temp1,columns=('lat','lng'))
df2 = pd.concat([df2, df_test], axis=1)
df2

수원시청


,updnLine,trainLineNm,statnNm,bstatnNm,arvlMsg2,arvlMsg3,barvlDt,arvlCd,lat,lng
6,하행,서동탄행 - 세류방면,수원,서동탄,전역 진입,화서,0,4,NaN,NaN
7,하행,고색행 - 고색방면,수원,고색,[2]번째 전역 (수원시청),수원시청,0,99,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.284894,126.9892
